In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cvxpy as cp
from scipy.optimize import minimize, Bounds, LinearConstraint
from sklearn.covariance import LedoitWolf
import statsmodels.api as sm
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
from math import sqrt
from itertools import product, combinations
from arch import arch_model as mgarch
from pandas.tseries.offsets import MonthEnd

Partie A

In [2]:
#Function for ARIMA model fitting and forecasting for returns
def fit_arma_models(rets, train_size=0.8, p_values=range(5), q_values=range(5)):
    # Split the data into training and testing sets
    train_size = int(len(rets) * train_size)
    train, test = rets[:train_size], rets[train_size:]

    # Create a matrix to store the best (p, q) values for each column
    best_params_matrix = np.zeros((rets.shape[1], 2), dtype=int)

    # Grid search to find the best (p, q) combination for each column
    best_rmse_values = [float('inf')] * rets.shape[1]

    for i in range(rets.shape[1]):
        for p, q in product(p_values, q_values):
            # Fit the ARIMA model on the training set for the current column
            model = ARIMA(train.iloc[:, i], order=(p, 0, q))
            results = model.fit()

            # Forecast the next 'n_periods' periods
            n_periods = len(test)
            forecast = results.get_forecast(steps=n_periods)

            # Extract the forecasted values and calculate RMSE
            forecast_df = forecast.predicted_mean
            rmse_value = sqrt(mean_squared_error(test.iloc[:, i], forecast_df))

            # Update the best parameters if a lower RMSE is found
            if rmse_value < best_rmse_values[i]:
                best_rmse_values[i] = rmse_value
                best_params_matrix[i, 0], best_params_matrix[i, 1] = p, q
    # Print the best (p, q) combinations for each column
    for i in range(rets.shape[1]):
        print(f"Best (p, q) for column {i}: ({best_params_matrix[i, 0]}, {best_params_matrix[i, 1]})")
    
    # Fit ARIMA models and return the results
    arma_models, arma_forecasts, arma_rmse_values = [], [], []

    for i in range(rets.shape[1]):
        # Extract the optimal (p, q) order for the current column
        p, q = best_params_matrix[i]

        # Fit the ARIMA model on the full dataset for the current column
        model = ARIMA(rets.iloc[:, i], order=(p, 0, q))
        results = model.fit()

        # Forecast the next period for the current column
        forecast = results.get_forecast(steps=1)
        forecast_df = forecast.predicted_mean

        # Calculate RMSE for the current column
        rmse_value = sqrt(mean_squared_error(rets.iloc[:, i][-1:], forecast_df))

        # Append the ARIMA model, forecast, and RMSE to the respective lists
      
        arma_forecasts.append(forecast_df)
        arma_rmse_values.append(rmse_value)

        # Naming the forecasts based on column names
    forecast_results = dict(zip(rets.columns, arma_forecasts))
    
    print( forecast_results, arma_rmse_values)
    return forecast_results, arma_rmse_values


In [ ]:

def find_best_alpha(rets):
    # Créer une plage de valeurs alpha à tester
    alpha_values = np.arange(0.01, 1.0, 0.1)

    # Initialiser des variables pour stocker les résultats
    meilleur_alpha_par_colonne = {}
    meilleure_metrique_par_colonne = {}

    # Effectuer une recherche sur la grille pour chaque colonne
    for colonne in rets.columns:
        meilleur_alpha = None
        meilleure_metrique = float('inf')  # Initialiser avec une valeur élevée
        
        for alpha in alpha_values:
            # Calculer la MME avec l'alpha actuel sur la colonne courante
            rendements_pond_exp = rets[colonne].ewm(span=1/alpha, adjust=False).mean()
            
            # Évaluer les performances en utilisant une métrique choisie (par exemple, l'erreur quadratique moyenne)
            # Ajustez cette partie en fonction de votre métrique d'évaluation spécifique
            metrique = ((rets[colonne] - rendements_pond_exp) ** 2).mean()
            
            # Mettre à jour le meilleur alpha si l'actuel donne de meilleures performances
            if metrique < meilleure_metrique:
                meilleur_alpha = alpha
                meilleure_metrique = metrique
        
        # Stocker le meilleur alpha et sa métrique correspondante pour la colonne courante
        meilleur_alpha_par_colonne[colonne] = meilleur_alpha
        meilleure_metrique_par_colonne[colonne] = meilleure_metrique

    # Retourner les meilleurs alphas et leurs métriques correspondantes pour chaque colonne
    return meilleur_alpha_par_colonne, meilleure_metrique_par_colonne


    print(f"Meilleur Alpha pour {colonne}: {alpha}")
    print(f"Meilleure Métrique pour {colonne}: {best_metrics[colonne]}")


In [18]:
#Fonction pour calculer les rendements espérés z 
def exp_rets(returns, alpha, method='moyenne', arma_order=None):
    """
    Objectif
    ---------- 
    Estimer les rendements attendus de nos portefeuilles industriels
    
    Paramètres
    ----------
    returns : DataFrame
        DataFrame qui contient les rendements des différents portefeuilles industriels.
    alpha : float
        Facteur de lissage utilisé si la méthode est 'moyenne_exp_ponderee'.
    method : str, optional
        Méthode à utiliser pour l'estimation des rendements attendus. 
        Les options sont 'moyenne_historique' (par défaut), 'MME' (Moyenne Mobile Exponentielle) ou 'arma'.
    arma_order : tuple, optional
        Ordre du modèle ARMA à ajuster, par exemple, (1, 1) pour un modèle ARMA(1, 1).
    
    Sortie
    ------
    Un tableau avec les rendements attendus pour chaque portefeuille industriel.
    """
    
    if method == 'moyenne':
        z = np.array(returns.mean())
        
    elif method == 'MME':
        find_best_alpha(returns)
        z = np.array(returns.ewm(span=1/alpha, adjust=False).mean().iloc[-1])
    
    elif method == 'arma' and arma_order:
        fit_arma_models(rets, train_size=0.8, p_values=range(5), q_values=range(5))
        z = np.array(forecast_results)
    
    # Annualiser le rendement 
    z = z * 12

    return z

In [4]:
#Fonction pour calculer la matrice de variance-covariance
def covariance_matrix(returns, method='LdW'):
    """
    Objectif
    ---------- 
    Estimer la matrice de covariance de nos portefeuilles industriels
    
    Paramètres
    ----------
    returns : DataFrame
        DataFrame qui contient les rendements des différents portefeuilles industriels.

    method : str, optional
        La méthode d'estimation de la matrice de covariance. Les options sont 'cov' (par défaut),
        'LdW' pour Ledoit-Wolf, 'Mgarch' pour MGARCH.
    """
    if method == 'cov':
        sigma = returns.cov()   # Covariance annualisée
        sigma = sigma.to_numpy()
    elif method == 'LdW':
        lw = LedoitWolf()
        lw.fit(returns)
        sigma = lw.covariance_  # Covariance annualisée
    
    elif method == 'Mgarch':
        # Convert 'returns' to a numpy matrix (assuming it's a DataFrame)
        rt = returns.to_numpy()

        # Specify distribution and fit the MGARCH model
        dist = 't'
        vol = mgarch.mgarch(dist)
        vol.fit(rt)

        # Specify the number of days for which you want to predict
        ndays = 1

        # Predict the conditional covariance matrix for the next day
        cov_nextday = vol.predict(ndays)

        # Extract the covariance matrix from the dictionary
        sigma = cov_nextday['cov']  # Covariance annualisée

    else:
        raise ValueError("Invalid method. Supported methods are 'cov', 'LdW', and 'Mgarch'.")

    return sigma * 12


Méthode de résolution analytique

In [5]:
def efficient_frontier_Analytique(exp_ret, sigma, R_cible, rf):
    """
    Optimisation de la frontière efficiente
    
    Paramètres:
    exp_ret : numpy array
        Matrice des rendements des actifs
    sigma : numpy array
        Matrice de covariance des rendements des actifs
    R_cible : float
        Rendement cible du portefeuille
    rf : float
        Taux sans risque
    
    Retours:
    Z_barre : numpy array
        Rendements attendus des actifs
    min_ret : float
        Rendement minimal du portefeuille
    min_std : float
        Écart-type minimal du portefeuille
    lambda_theta : numpy array
        Multiplicateurs de Lagrange
    w_optimal : numpy array
        Poids optimaux du portefeuille
    sharpe_ratios : numpy array
        Ratios de Sharpe pour tous les points de la frontière efficiente
    wi_optimal : numpy array
        Poids optimaux du portefeuille pour la frontière efficiente
    rendements_optimaux : numpy array
        Rendements optimaux pour la frontière efficiente
    rendement_optimal : float
        Rendement du portefeuille optimal
    volatilites_optimales : numpy array
        Volatilités optimales des portefeuilles pour la frontière efficiente
    rendements_optimaux_etendus : numpy array
        Rendements optimaux étendus pour la frontière complète
    volatilites_optimales_etendues : numpy array
        Volatilités optimales étendues des portefeuilles pour la frontière complète
    tangent_portfolio_return : float
        Rendement du portefeuille tangent
    tangent_portfolio_volatility : float
        Volatilité du portefeuille tangent
    CML_ret : numpy array
        Rendements attendus pour la ligne du marché des capitaux (CML)
    CML_std : numpy array
        Écarts types pour la ligne du marché des capitaux (CML)
    """

    Z_barre = exp_ret  # Rendements attendus des actifs
    ones = np.ones(len(Z_barre))  # Vecteur de uns

    # Inversion de la matrice de covariance
    Sigma_inv = np.linalg.inv(sigma)

    # Construction de la matrice A et du vecteur b
    A = np.array([
        [np.dot(ones, np.dot(Sigma_inv, ones)), np.dot(ones, np.dot(Sigma_inv, Z_barre))],
        [np.dot(Z_barre, np.dot(Sigma_inv, ones)), np.dot(Z_barre, np.dot(Sigma_inv, Z_barre))]
    ])

    b = np.array([1, R_cible])

    # Inversion de la matrice A
    A_inv = np.linalg.inv(A)

    # Résolution pour obtenir les multiplicateurs de Lagrange
    lambda_theta = np.dot(A_inv, b)

    # Calcul des poids optimaux du portefeuille
    w_optimal = lambda_theta[0] * np.dot(Sigma_inv, ones) + lambda_theta[1] * np.dot(Sigma_inv, Z_barre)

    # Intervalle des rendements cibles pour la frontière efficiente
    rendements_cibles_efficients = np.linspace(min(Z_barre), max(Z_barre), 100)

    # Calcul des volatilités et des rendements pour la frontière efficiente
    volatilites_optimales = []
    rendements_optimaux = []

    for R_cible in rendements_cibles_efficients:
        b = np.array([1, R_cible])
        lambda_theta = np.dot(A_inv, b)
        wi_optimal = lambda_theta[0] * np.dot(Sigma_inv, ones) + lambda_theta[1] * np.dot(Sigma_inv, Z_barre)
        rendement_optimal = np.dot(wi_optimal, Z_barre)
        volatilite_optimale = np.sqrt(np.dot(wi_optimal.T, np.dot(sigma, wi_optimal)))
        rendements_optimaux.append(rendement_optimal)
        volatilites_optimales.append(volatilite_optimale)

    # Étendre l'intervalle des rendements cibles pour la frontière complète
    rendements_cibles_etendus = np.linspace(min(Z_barre) - 0.015, max(Z_barre) + 0.015, 200)

    # Calcul des volatilités et des rendements pour la frontière complète
    volatilites_optimales_etendues = []
    rendements_optimaux_etendus = []

    for R_cible in rendements_cibles_etendus:
        b = np.array([1, R_cible])
        lambda_theta = np.dot(A_inv, b)
        wi_optimal = lambda_theta[0] * np.dot(Sigma_inv, ones) + lambda_theta[1] * np.dot(Sigma_inv, Z_barre)
        rendement_optimal = np.dot(wi_optimal, Z_barre)
        volatilite_optimale = np.sqrt(np.dot(wi_optimal.T, np.dot(sigma, wi_optimal)))
        rendements_optimaux_etendus.append(rendement_optimal)
        volatilites_optimales_etendues.append(volatilite_optimale)

    # Calcul des poids optimaux du portefeuille avec la variance minimale
    w_minvar = np.dot(Sigma_inv, ones) / np.dot(ones.T, np.dot(Sigma_inv, ones))

    # Calcul de la variance minimale du portefeuille
    min_var = 1 / np.dot(ones.T, np.dot(Sigma_inv, ones))

    # Calcul de l'écart-type minimal du portefeuille
    min_std = np.sqrt(min_var)

    # Calcul du rendement du portefeuille avec la variance minimale
    min_ret = np.dot(exp_ret, w_minvar)

    print(f"Portefeuille avec variance minimale: {w_minvar}, Variance minimale: {min_var}, Écart-type minimal: {min_std}, Rendement minimal: {min_ret}")

    # Calcul de la pente de la Ligne du Marché des Capitaux (CML)
    CML_slope = (max(rendements_optimaux) - rf) / max(volatilites_optimales)

    # Générer les écarts types pour la Ligne du Marché des Capitaux (CML)
    CML_std = np.linspace(0, max(volatilites_optimales), 100)

    # Calcul des rendements correspondants pour la Ligne du Marché des Capitaux (CML)
    CML_ret = rf + CML_slope * CML_std

    # Calcul des ratios de Sharpe pour tous les points de la frontière efficiente
    sharpe_ratios = [(r - rf) / v for r, v in zip(rendements_optimaux, volatilites_optimales)]

    # Trouver l'indice du portefeuille avec le ratio de Sharpe maximal
    max_sharpe_index = np.argmax(sharpe_ratios)

    # Extraire le rendement et la volatilité du portefeuille tangent
    tangent_portfolio_return = rendements_optimaux[max_sharpe_index]
    tangent_portfolio_volatility = volatilites_optimales[max_sharpe_index]


    return Z_barre, min_ret, min_std, lambda_theta, w_optimal, sharpe_ratios, wi_optimal, rendements_optimaux, rendement_optimal, volatilites_optimales, rendements_optimaux_etendus, volatilites_optimales_etendues, tangent_portfolio_return, tangent_portfolio_volatility, CML_ret, CML_std 

In [6]:
def plot_efficient_frontier_an(ret, sigma, rf, include_CML=False, asset_names = None):
    """
    Trace la frontière efficiente complète avec les portefeuilles industriels.
    
    Paramètres :
    ret : numpy array
        Matrice des rendements des actifs
    sigma : numpy array
        Matrice de covariance des rendements des actifs
    rf : float
        Taux sans risque
    include_CML : booléen, optionnel
        Indique s'il faut inclure la ligne du marché des capitaux (CML) dans le graphique. Par défaut, c'est True.
    asset_names : list, optionnel
        Liste des noms des actifs. Par défaut, c'est None.
    """
    # Appel de la fonction efficient_frontier_optimization
    Z_barre, min_ret, min_std, lambda_theta, w_optimal, sharpe_ratios, wi_optimal, rendements_optimaux, rendement_optimal, volatilites_optimales, rendements_optimaux_etendus, volatilites_optimales_etendues, tangent_portfolio_return, tangent_portfolio_volatility, CML_ret, CML_std = efficient_frontier_Analytique(ret, sigma, R_cible, rf)
    
    if not include_CML:
        # Tracer la frontière complète et la frontière efficiente
        plt.figure(figsize=(10, 6))
        plt.plot(volatilites_optimales_etendues, rendements_optimaux_etendus, label='Mean-variance locus', color='grey')
        plt.plot(volatilites_optimales, rendements_optimaux, label='Efficient frontier', color='blue')
        plt.scatter(np.sqrt(np.diag(sigma)), Z_barre, color='purple', label='Industries portfolios')
        plt.scatter(min_std, min_ret, color='red', label='Minimum Variance Portfolio', edgecolors='black', s=100, zorder=5) 
        if asset_names:
            for i, name in enumerate(asset_names):
                plt.scatter(np.sqrt(sigma[i,i ]), Z_barre[i], color = 'red', label=name, marker = 'o')
                plt.text(np.sqrt(sigma[i,i]), Z_barre[i], name, fontsize=12, ha='right', va = 'bottom') 
        plt.title('Mean-Variance Locus with Efficient Frontier')
        plt.xlabel('Standard Deviation (Volatility)')
        plt.ylabel('Expected Return (Mean)')
        plt.legend(loc='upper left')
        plt.grid(True)
        plt.xlim(0.10, 0.23)
        plt.ylim(0, 0.23)  
        plt.show()
    else:
        # Tracer la frontière complète et la frontière efficiente
        plt.figure(figsize=(10, 6))
        plt.plot(volatilites_optimales_etendues, rendements_optimaux_etendus, label='Mean-variance locus', color='black')
        plt.plot(volatilites_optimales, rendements_optimaux, label='Efficient frontier', color='blue')
        plt.plot(CML_std, CML_ret, label='Capital Market Line', color='black', linestyle='--')
        plt.scatter(np.sqrt(np.diag(sigma)), Z_barre, color='purple', label='Industries portfolios')
        plt.scatter(tangent_portfolio_volatility, tangent_portfolio_return, color='blue', marker='o', s=100, label='Tangent Portfolio', edgecolors='black', zorder=5)
        plt.scatter(min_std, min_ret, color='red', label='Minimum Variance Portfolio', edgecolors='black', s=100, zorder=5)
        if asset_names:
            for i, name in enumerate(asset_names):
                plt.scatter(np.sqrt(sigma[i,i ]), Z_barre[i], color = 'red', label=name, marker = 'o')
                plt.text(np.sqrt(sigma[i,i]), Z_barre[i], name, fontsize=12, ha='right', va = 'bottom') 
        plt.title('Mean-Variance Locus with Efficient Frontier')
        plt.xlabel('Standard Deviation (Volatility)')
        plt.ylabel('Expected Return (Mean)')
        plt.legend(loc='upper left')
        plt.grid(True)
        plt.xlim(0.10, 0.23)
        plt.ylim(0, 0.23)       
        plt.show()
        print("Poids optimaux du portefeuille tangent :")
        print(["Cnsmr", "Manuf", "HiTech", "Hlth", "Other"], wi_optimal) #Poids optimaux du portefeuille
        print("Rendement du portefeuille tangent :", tangent_portfolio_return)
        print("Volatilité du portefeuille tangent :", tangent_portfolio_volatility)

   
    print("Lambda (lambda_theta[0]) :", lambda_theta[0]) #Lambda multiplicateur de Lagrange
    print("Theta (lambda_theta[1]) :", lambda_theta[1]) #Theta multiplicateur de Lagrange
    



In [7]:
# Function to find the index of the portfolio with the maximum Sharpe ratio
def max_sharpe_ratio(ret, sigma, rf):
    # Appel de la fonction efficient_frontier_optimization
    Z_barre, min_ret, min_std, lambda_theta, w_optimal, sharpe_ratios, wi_optimal, rendements_optimaux, rendement_optimal, volatilites_optimales, rendements_optimaux_etendus, volatilites_optimales_etendues, tangent_portfolio_return, tangent_portfolio_volatility, CML_ret, CML_std = efficient_frontier_Analytique(ret, sigma, R_cible, rf)

    
    max_sharpe_index = np.argmax(sharpe_ratios)

    # Extract the weights of the tangent portfolio
    tangent_portfolio_weights = wi_optimal 

    # Calculate the mean (expected return) of the tangent portfolio
    tangent_portfolio_mean = rendements_optimaux[max_sharpe_index]

    # Calculate the variance of the tangent portfolio
    tangent_portfolio_variance = volatilites_optimales[max_sharpe_index] ** 2
    tangent_portfolio_volatility = volatilites_optimales[max_sharpe_index]

    # Print or use these values as needed
    print("Tangent Portfolio Mean:", tangent_portfolio_mean)
    print("Tangent Portfolio Variance:", tangent_portfolio_variance)
    print("Tangent Portfolio Volatility:", tangent_portfolio_volatility)
    print("Tangent Portfolio Weights:", tangent_portfolio_weights)

    # Plot the Sharpe ratio against portfolio volatility
    plt.figure(figsize=(10, 6))
    plt.scatter(volatilites_optimales, sharpe_ratios, color='orange', label='Sharpe Ratio')

    # Highlight the tangent portfolio
    plt.scatter(tangent_portfolio_volatility, sharpe_ratios[max_sharpe_index], color='green', marker='o', s=100, label='Tangent Portfolio')

    # Add labels and title
    plt.title('Sharpe Ratio vs Portfolio Volatility')
    plt.xlabel('Portfolio Volatility')
    plt.ylabel('Sharpe Ratio')
    plt.legend(loc='upper left')
    plt.grid(True)

    # Show the plot
    plt.show()
    
    print('Sharpe Ratios:', sharpe_ratios)
    print('Maximum Sharpe Ratio:', max(sharpe_ratios))

Méthode de résolution numérique

In [15]:

def minimum_variance_portfolio(z, sigma, short_selling=True):
    """
    Calculate the minimum variance portfolio weights with short selling.

    Parameters
    ----------
    z : np.array
        Rendements attendus pour chaque actif.
    sigma : np.array
        Matrice de covariance.

    Returns
    -------
    np.array
        Poids du portefeuille minimisant la variance.
    """
    n = len(z)
    w = cp.Variable(n)
    portfolio_volatility = cp.quad_form(w, sigma)

    # Contraintes du portefeuille
    if short_selling:
        contraintes = [cp.sum(w) == 1]
    else:
        contraintes = [cp.sum(w) == 1, w >= 0]

    # Problème d'optimisation du portefeuille
    probleme = cp.Problem(cp.Minimize(portfolio_volatility), contraintes)

    # Résoudre le problème
    probleme.solve()

    return w.value           


In [16]:
def efficient_frontier_numerique(exp_ret, sigma, sim, R_cible, rf, rf_asset = False, short_selling = False, asset_names = None):
    """
    Fonction pour tracer la frontière efficiente moyenne-variance avec les actifs individuels et le portefeuille tangent.

    Paramètres
    ----------
    exp_ret : np.array
        Rendements attendus pour chaque actif.
    sigma : np.array
        Matrice de covariance.
    sim : int
        Nombre de simulations.
    rf : float
        Taux sans risque.
    rf_asset : bool, facultatif
        Inclure l'actif sans risque, par défaut False.
    short_selling : bool, facultatif
        Autoriser la vente à découvert ou non
    asset_names : list, facultatif
        Noms des actifs pour l'étiquetage du graphique.
    
    Renvoie
    -------
    rendements_optimaux : np.array
        Rendements optimaux pour la frontière efficiente.
    volatilites_optimales : np.array
        Volatilités optimales des portefeuilles pour la frontière efficiente.
    tangent_optimal_weights : np.array
        Poids optimaux du portefeuille tangent.
    tangent_portfolio_return : float
        Rendement du portefeuille tangent.
    tangent_portfolio_volatility : float
        Volatilité du portefeuille tangent.
    CML_returns : np.array
        Rendements attendus pour la ligne du marché des capitaux (CML).
    CML_volatilities : np.array
        Écarts types pour la ligne du marché des capitaux (CML).
    sharpe_ratios : np.array
        Ratios de Sharpe pour tous les points de la frontière efficiente.
    max_sharpe_index : int
        Indice du portefeuille avec le ratio de Sharpe maximal.
    poids_optimaux : np.array
        Poids optimaux du portefeuille pour la frontière efficiente.
    min_volatility : float
        Volatilité minimale.
    min_volatility_return : float
        Rendement associé à la volatilité minimale.
    minportfolio_weights : np.array
        Poids du portefeuille variance minimale.
    
    """
    Z_barre = exp_ret  # Rendements attendus des actifs
    ones = np.ones(len(Z_barre))  # Vecteur de uns
    
    

    # Fonction objectif : Variance du portefeuille
    def minportfolio_variance(W, sigma):
        return W.T @ (sigma @ W)
   
    
    # Poids initiaux : répartition uniforme
    W = np.ones(len(Z_barre)) * (1.0 / len(Z_barre))

    # Fonction d'optimisation
    def optimize(func, W, sigma, target_return, short_selling=False):
        if not short_selling:
            opt_constraints = ({'type': 'eq', 'fun': lambda W: ones @ W.T - 1},  # La somme des poids doit être égale à 1
                               {'type': 'eq', 'fun': lambda W: W.T @ Z_barre - target_return},  # Le rendement attendu doit être égal au rendement cible
                               {'type': 'ineq', 'fun': lambda W: W})  # No short selling constraint
            bounds = Bounds(0, 1)

        else: # Contraintes
            opt_constraints = ({'type': 'eq', 'fun': lambda W: ones @ W.T - 1},  # La somme des poids doit être égale à 1
                               {'type': 'eq', 'fun': lambda W: W.T @ Z_barre - target_return})  # Le rendement attendu doit être égal au rendement cible
            bounds = None

        # Optimisation
        optimal_weights = minimize(func, W, args=(sigma), method='SLSQP', constraints=opt_constraints, bounds=bounds)

        return optimal_weights.x


    # Générer une série de rendements cibles
    rendements_cibles = np.linspace(min(Z_barre) - 0.015, max(Z_barre) + 0.015, 100)

    # Initialiser les listes pour stocker les volatilités et rendements optimaux
    volatilites_optimales = []
    rendements_optimaux = []

    # Itérer sur les rendements cibles pour optimiser les poids du portefeuille
    for R_cible in rendements_cibles:
        W = np.ones(len(Z_barre)) * (1.0 / len(Z_barre))  # Réinitialiser les poids initiaux pour chaque itération
        poids_optimaux = optimize(minportfolio_variance, W, sigma, R_cible, short_selling)
        var_optimale = minportfolio_variance(poids_optimaux, sigma)
        rendement_optimal = np.dot(poids_optimaux, Z_barre)

        volatilites_optimales.append(np.sqrt(var_optimale))
        rendements_optimaux.append(rendement_optimal)
    # Find the index of the portfolio with the minimum volatility
    min_volatility_index = np.argmin(volatilites_optimales)

    # Extract the return associated with the minimum volatility
    min_volatility_return = rendements_optimaux[min_volatility_index]

    print("Return associated with minimum volatility:", min_volatility_return)
    min_volatility = min(volatilites_optimales)
    print("Std minimale:", min_volatility)

    #Find the weights of the minimum variance portfolio
    W = np.ones(len(Z_barre)) * (1.0 / len(Z_barre))  # Réinitialiser les poids initiaux pour chaque itération
    minportfolio_weights = minimum_variance_portfolio(Z_barre, sigma, short_selling)
    print("Poids du portefeuille variance minimale:", minportfolio_weights)
  
    # Calculate the CML equation
    CML_returns = np.linspace(rf, max(rendements_optimaux), 100)
    CML_volatilities = (CML_returns - rf) / ((max(rendements_optimaux) - rf) / max(volatilites_optimales))

    # Calculate Sharpe Ratios for all points on the efficient frontier
    sharpe_ratios = [(r - rf) / v for r, v in zip(rendements_optimaux, volatilites_optimales)]

    # Find the index of the portfolio with the maximum Sharpe ratio
    max_sharpe_index = np.argmax(sharpe_ratios)

    #Poids optimaux du portefeuille tangent
    tangent_optimal_weights = optimize(minportfolio_variance, W, sigma, R_cible, short_selling)
    # Extract the return and volatility of the tangent portfolio
    tangent_portfolio_return = rendements_optimaux[max_sharpe_index]
    tangent_portfolio_volatility = volatilites_optimales[max_sharpe_index]
    
    # Extract the volatility of the tangent portfolio using its optimal weights
    #tangent_portfolio_volatility = np.sqrt(minportfolio_variance(tangent_optimal_weights, sigma))



    # Générer une série de rendements cibles
    rendements_cibles = np.linspace(min(Z_barre) - 0.015, max(Z_barre) + 0.015, 100)

    
    if not rf_asset:
        # Tracer la frontière de variance moyenne
        plt.figure(figsize=(10, 6))
        plt.plot(volatilites_optimales, rendements_optimaux, 'b-', label='Mean-Variance Locus (Rf not included)')
        if asset_names:
            for i, name in enumerate(asset_names):
                plt.scatter(np.sqrt(sigma[i,i ]), Z_barre[i], color = 'red', label=name, marker = 'o')
                plt.text(np.sqrt(sigma[i,i]), Z_barre[i], name, fontsize=12, ha='right', va = 'bottom') 
        plt.scatter(np.sqrt(np.diag(sigma)), Z_barre, color='red', label='Individual Portfolios')
        plt.scatter(min_volatility, min_volatility_return, color='green', marker='o', s=150, label='Minimum Variance Portfolio', edgecolors='black')
        plt.title('Mean-Variance Locus of Industry Portfolios')
        plt.xlabel('Volatility (Standard Deviation)')
        plt.ylabel('Expected Return')
        plt.legend(loc='upper left')
        plt.grid(True)
    else:
        # Tracer la frontière de variance moyenne
        plt.figure(figsize=(10, 6))
        plt.plot(volatilites_optimales, rendements_optimaux, 'b-', label='Mean-Variance Locus (Rf included)')
        if asset_names:
            for i, name in enumerate(asset_names):
                plt.scatter(np.sqrt(sigma[i,i ]), Z_barre[i], color = 'red', label=name, marker = 'o')
                plt.text(np.sqrt(sigma[i,i]), Z_barre[i], name, fontsize=12, ha='right', va = 'bottom') 
        plt.plot(CML_volatilities, CML_returns, 'b--', label='Capital Market Line')
        plt.scatter(np.sqrt(np.diag(sigma)), Z_barre, color='red', label='Individual Portfolios')
        plt.scatter(tangent_portfolio_volatility, tangent_portfolio_return, color='red', marker='o', s=100, label='Tangent Portfolio')
        plt.scatter(min_volatility, min_volatility_return, color='green', marker='o', s=150, label='Minimum Variance Portfolio', edgecolors='black')
        plt.title('Mean-Variance Locus of Industry Portfolios')
        plt.xlabel('Volatility (Standard Deviation)')
        plt.ylabel('Expected Return')
        plt.legend(loc='upper left')
        plt.grid(True)
        print("Poids optimaux du portefeuille tangent", tangent_optimal_weights)
        print("Rendement du portefeuille tangent:", tangent_portfolio_return)
        print("Volatilité du portefeuille tangent:", tangent_portfolio_volatility)


    return rendements_optimaux, volatilites_optimales, tangent_optimal_weights, tangent_portfolio_return, tangent_portfolio_volatility, CML_returns, CML_volatilities, sharpe_ratios, max_sharpe_index, poids_optimaux, min_volatility, min_volatility_return, minportfolio_weights

In [17]:
def plot_efficient_frontier_num_tangent(exp_ret, sigma, sim, R_cible, rf, rf_asset=False, short_selling=False):
    # ... function implementation ...
    rendements_optimaux, volatilites_optimales, tangent_optimal_weights, tangent_portfolio_return, tangent_portfolio_volatility, CML_returns, CML_volatilities, sharpe_ratios, max_sharpe_index, poids_optimaux, min_volatility, min_volatility_return, minportfolio_weights  = efficient_frontier_numerique(exp_ret, sigma, sim, R_cible, rf, rf_asset, short_selling)
    
    # Plot the efficient frontier
    plt.plot(volatilites_optimales, rendements_optimaux, marker='o', linestyle='-')

    # Plot the tangent portfolio point
    plt.scatter(tangent_portfolio_volatility, tangent_portfolio_return, color='red', marker='o', s=100, label='Tangent Portfolio')

    #   Plot the Sharpe ratio against portfolio volatility
    plt.figure(figsize=(10, 6))
    
    plt.scatter(volatilites_optimales, sharpe_ratios, color='orange', label='Sharpe Ratio')

    # Highlight the tangent portfolio
    plt.scatter(tangent_portfolio_volatility, sharpe_ratios[max_sharpe_index], color='green', marker='o', s=100, label='Tangent Portfolio')

    # Add labels and title
    plt.title('Sharpe Ratio vs Portfolio Volatility')
    plt.xlabel('Portfolio Volatility')
    plt.ylabel('Sharpe Ratio')
    plt.legend(loc='upper left')
    plt.grid(True)

    # Show the plot
    plt.show()

    print('Sharpe Ratios:', sharpe_ratios)
    print('Max sharpe ratios:', max(sharpe_ratios))
    print('Tangent Portfolio Return:', tangent_portfolio_return)
    print('Tangent Portfolio Volatility:', tangent_portfolio_volatility)
    print("Weights of the tangent portfolio:", tangent_optimal_weights)
    
   

# Partie B

# Bootstrap

In [11]:
def bootstrap_A(ret, R_cible, rf, methode_moy, N_boot, cov_lw=True, include_CML=False):
    num_bootstrap_samples = N_boot
    all_volatilities = []
    all_returns = []
    all_tangent_portfolio_weights = []
    all_tangent_portfolio_rets = []
    all_tangent_portfolio_variances = []
    all_tangent_portfolio_sharpe_ratios = []
    
    for _ in range(num_bootstrap_samples):
        sample = ret.sample(n=len(ret), replace=True)
        ret_moy = exp_rets(sample, 0.3, method=methode_moy)
        
        if cov_lw:
            sigma_boot = covariance_matrix(sample)
        else:
            sigma_boot = np.cov(sample, rowvar=False) * 12
        ones = np.ones(len(ret_moy))  
        Sigma_inv = np.linalg.inv(sigma_boot)

        A = np.array([
            [np.dot(ones, np.dot(Sigma_inv, ones)), np.dot(ones, np.dot(Sigma_inv, ret_moy))],
            [np.dot(ret_moy, np.dot(Sigma_inv, ones)), np.dot(ret_moy, np.dot(Sigma_inv, ret_moy))]
        ])
        b = np.array([1, R_cible])
        A_inv = np.linalg.inv(A)

        rendements_cibles_etendus = np.linspace(min(ret_moy) - 0.015, max(ret_moy) + 0.015, 200)
        volatilites_optimales_etendues = []
        rendements_optimaux_etendus = []
        for R_cible in rendements_cibles_etendus:
            b = np.array([1, R_cible])
            lambda_theta = np.dot(A_inv, b)
            wi_optimal = lambda_theta[0] * np.dot(Sigma_inv, ones) + lambda_theta[1] * np.dot(Sigma_inv, ret_moy)
            rendement_optimal = np.dot(wi_optimal, ret_moy)
            volatilite_optimale = np.sqrt(np.dot(wi_optimal.T, np.dot(sigma_boot, wi_optimal)))
            rendements_optimaux_etendus.append(rendement_optimal)
            volatilites_optimales_etendues.append(volatilite_optimale)

        all_volatilities.append(volatilites_optimales_etendues)
        all_returns.append(rendements_optimaux_etendus)

        # Calculate tangent portfolio weights, means, and variances
        tangent_portfolio_weights = np.linalg.inv(sigma_boot) @ (ret_moy - rf*ones) / (A[0, 1] - A[0, 0] * rf)
        tangent_portfolio_return = np.dot(tangent_portfolio_weights, ret_moy.T)
        tangent_portfolio_variance = np.dot(tangent_portfolio_weights.T, np.dot(sigma_boot, tangent_portfolio_weights))

        all_tangent_portfolio_weights.append(tangent_portfolio_weights)
        all_tangent_portfolio_rets.append(tangent_portfolio_return)
        all_tangent_portfolio_variances.append(tangent_portfolio_variance)

        # Calculate Sharpe ratio for the tangent portfolio
        tangent_portfolio_sharpe_ratio = (tangent_portfolio_return - rf * 12) / np.sqrt(tangent_portfolio_variance)
        all_tangent_portfolio_sharpe_ratios.append(tangent_portfolio_sharpe_ratio)
        max_sharpe_tangent_portfolio_index = np.argmax(all_tangent_portfolio_sharpe_ratios)
        
        
        tangent_weights = all_tangent_portfolio_weights[max_sharpe_tangent_portfolio_index]
        mean_tangent_returns = np.mean(all_tangent_portfolio_rets)
        mean_tangent_var = np.mean(all_tangent_portfolio_variances)
        mean_tangent_sharpe = np.mean(all_tangent_portfolio_sharpe_ratios)

    if not include_CML:
        plt.figure(figsize=(10, 6))
        for volatilities, returns in zip(all_volatilities, all_returns):
            plt.plot(volatilities, returns, 'b-', alpha=0.3)

        plt.title('Mean-Variance Locus with Efficient Frontier (100 Bootstraps)')
        plt.xlabel('Volatility')
        plt.ylabel('Expected Return')
        plt.show()

    else:
        plt.figure(figsize=(10, 6))
        for volatilities, returns in zip(all_volatilities, all_returns):
            plt.plot(volatilities, returns, color='blue', alpha=0.4)  # Alpha for transparency
            CML_slope = (max(returns) - rf) / max(volatilities)
            CML_std = np.linspace(0, max(volatilities), 100)
            CML_ret = rf + CML_slope * CML_std
            plt.plot(CML_std, CML_ret, label='Capital Market Line', color='black', linestyle='--')

        plt.title('Mean-Variance Locus with Efficient Frontier and CML (100 Bootstraps)')
        plt.xlabel('Standard Deviation (Volatility)')
        plt.ylabel('Expected Return (Mean)')
        plt.xlim(0)
        plt.show()
        
    return (f"Tangent weights: {tangent_weights}", 
            f"Mean tangent returns: {mean_tangent_returns}", 
            f"Mean tangent variance: {mean_tangent_var}", 
            f"Mean tangent Sharpe ratio: {mean_tangent_sharpe}")


In [12]:
def bootstrap_N(ret, R_cible, rf, methode_moy, N_boot, cov_lw=True, include_CML=False):
    num_bootstrap_samples = N_boot
    all_volatilities = []
    all_returns = []
    tangent_weights_list = []
    tangent_var_list = []
    tangent_ret_list = []
    sharpe_ratio_list = []
    
    for _ in range(num_bootstrap_samples):
        sample = ret.sample(n=len(ret), replace=True)
        ret_moy = exp_rets(sample, 0.94, method=methode_moy)
        
        if cov_lw:
            sigma_boot = covariance_matrix(sample)
        else:
            sigma_boot = np.cov(sample, rowvar=False) * 12
        ones = np.ones(len(ret_moy))
        
        # Fonction objectif : Variance du portefeuille
        def minportfolio_variance(W, sigma_boot):
            return W.T @ (sigma_boot @ W)
        
        # Fonction objectif : Négatif du ratio de Sharpe
        def negativeSR(w, sigma_boot):
            R = np.sum(ret_moy * w)
            V = np.sqrt(np.dot(w.T, np.dot(sigma_boot, w)))
            SR = R / V
            return -SR

    
        # Poids initiaux : répartition uniforme
        W = np.ones(len(ret_moy)) * (1.0 / len(ret_moy))
        
        # Fonction d'optimisation
        def optimize(func, W, sigma_boot, target_return):
            # Contraintes
            opt_constraints = ({'type': 'eq', 'fun': lambda W: ones @ W.T - 1},  # Somme des poids égale à 1
                               {'type': 'eq', 'fun': lambda W: W.T @ ret_moy - target_return},  # Rendement attendu égal à rendement cible
                               {'type': 'ineq', 'fun': lambda W: W})  # Pas de vente à découvert

            # Optimisation
            optimal_weights = minimize(func, W, args=(sigma_boot), method='trust-constr', constraints=opt_constraints)

            return optimal_weights.x
    
# Générer une série de rendements cibles
        rendements_cibles = np.linspace(min(ret_moy) - 0.015, max(ret_moy) + 0.015, 100)

        # Initialiser les listes pour stocker les volatilités et rendements optimaux
        volatilites_optimales = []
        rendements_optimaux = []

        # Itérer sur les rendements cibles pour optimiser les poids du portefeuille
        for R_cible in rendements_cibles:
            W = np.ones(len(ret_moy)) * (1.0 / len(ret_moy))  # Réinitialiser les poids initiaux pour chaque itération
            poids_optimaux = optimize(minportfolio_variance, W, sigma_boot, R_cible)
            var_optimale = minportfolio_variance(poids_optimaux, sigma_boot)
            rendement_optimal = np.dot(poids_optimaux, ret_moy)

            volatilites_optimales.append(np.sqrt(var_optimale))
            rendements_optimaux.append(rendement_optimal)
            
        # Store data for efficient frontier
        all_volatilities.append(volatilites_optimales)
        all_returns.append(rendements_optimaux)
        
        tangent_weights = optimize(negativeSR, W, sigma_boot, R_cible)
        tangent_var = np.dot(tangent_weights.T, np.dot(sigma_boot, tangent_weights))
        tangent_ret = np.dot(tangent_weights, ret_moy.T)
        tangent_sharpe_ratio = (tangent_ret - rf) / np.sqrt(tangent_var)
        max_tangent_sharpe_ratio = np.argmax(tangent_sharpe_ratio)
        
         # Stockez les valeurs dans les listes
        tangent_weights_list.append(tangent_weights)
        tangent_var_list.append(tangent_var)
        tangent_ret_list.append(tangent_ret)
        sharpe_ratio_list.append(tangent_sharpe_ratio)
        
        tangent_weights_max_SR = tangent_weights_list[max_tangent_sharpe_ratio]
        mean_tangent_returns = np.mean(tangent_ret_list)
        mean_tangent_var = np.mean(tangent_var_list)
        mean_tangent_sharpe = np.mean(sharpe_ratio_list)
        
    if not include_CML:
        plt.figure(figsize=(10, 6))
        for volatilities, returns in zip(all_volatilities, all_returns):
            plt.plot(volatilities, returns, 'b-', alpha=0.3)

        plt.xlabel('Volatility')
        plt.ylabel('Expected Return')
        plt.title('Efficient Frontier and Tangent Portfolio')
        plt.show()

    else:
        plt.figure(figsize=(10, 6))
        for volatilities, returns in zip(all_volatilities, all_returns):
            plt.plot(volatilities, returns, color='blue', alpha=0.4)  # Alpha for transparency
            CML_slope = (max(returns) - rf) / max(volatilities)
            CML_std = np.linspace(0, max(volatilities), 100)
            CML_ret = rf + CML_slope * CML_std
            plt.plot(CML_std, CML_ret, label='Capital Market Line', color='black', linestyle='--')

        plt.title('Mean-Variance Locus with Efficient Frontier and CML (100 Bootstraps)')
        plt.xlabel('Standard Deviation (Volatility)')
        plt.ylabel('Expected Return (Mean)')
        plt.xlim(0)
        plt.show()
        
    return tangent_weights_max_SR, mean_tangent_returns, mean_tangent_var, mean_tangent_sharpe


In [13]:
#Fonction pour question Partie 2 #2
def best_comb_portefeuille(ret, N, min_num_assets, max_num_assets, min_weight, target_return, R_cible_basse, R_cible_haute, portfolio_names):
    ret_moy = exp_rets(ret, 0.94, method='moyenne')
    cov_lw = covariance_matrix(ret)
    ones = np.ones(len(ret_moy))
    ## Fonction objectif : Variance du portefeuille
    def portfolio_variance(W,Sigma):
        return W.T @ (Sigma @ W)

    # Poids initiaux : répartition uniforme
    W = np.ones(len(ret_moy)) * (1.0 / len(ret_moy))

    # Fonction d'optimisation
    def optimize(func, W, Sigma, target_return):

        # Contraintes
        opt_constraints = ({'type': 'eq', 'fun': lambda W: ones @ W.T - 1},  # La somme des poids doit être égale à 1
                        {'type': 'eq', 'fun': lambda W: W.T @ ret_moy - target_return})  # Le rendement attendu doit être égal au rendement cible

        # Optimisation
        optimal_weights = minimize(func, W, args=( Sigma), method='trust-constr', constraints=opt_constraints)

        return optimal_weights.x

    # Appel de la fonction d'optimisation
    optimal_weights = optimize(portfolio_variance, W, cov_lw, target_return)

    
    
    # Résultats initiaux
    best_variance = np.inf
    best_weights = None
    best_combination = None

    # Générer toutes les combinaisons de 3 portefeuilles parmi 5
    for combination in combinations(range(N), 3):
        Sigma_sub = cov_lw[np.ix_(combination, combination)]
        Z_barre_sub = ret_moy[np.array(combination)]  # Assurez-vous que Z_barre_sub est correctement défini

        initial_weights = np.ones(len(combination)) / len(combination)

        # Définissez les contraintes en utilisant Z_barre_sub
        current_constraints = [
            {'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1},  # Les poids doivent sommer à 1
            {'type': 'ineq', 'fun': lambda weights: np.dot(weights, Z_barre_sub) - R_cible_basse},
            {'type': 'ineq', 'fun': lambda weights: R_cible_haute - np.dot(weights, Z_barre_sub)}  # Utilisez np.dot ici pour la compatibilité dimensionnelle
        ]

        # Lancer l'optimisation avec Sigma_sub et Z_barre_sub
        result = minimize(portfolio_variance, initial_weights, args=(Sigma_sub,), method='trust-constr', constraints=current_constraints)

        if result.success and result.fun < best_variance:
            best_variance = result.fun
            best_weights = result.x
            best_combination = [portfolio_names[i] for i in combination]  # Mappage des indices aux noms de portefeuilles

    # Affichage des résultats
    if best_combination is not None:
        print("Meilleure combinaison d'actifs:", best_combination)
    

    return best_combination, best_weights, best_variance